In [1]:
# Импорт необходимых библиотек
import os
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.registry import default_registry
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple

# Обучение агента в Unity ML-Agents
В этом ноутбуке показан пример запуска обучения агента в пользовательской среде Unity, расположенной в этой папке. Для этого используется пакет `mlagents` и Python API.

In [2]:
# Путь к вашему Unity окружению (укажите путь к .exe, если экспортировано, либо используйте Editor для запуска из редактора)
# env_path = None  # Если среда запускается из редактора Unity, иначе укажите путь к .exe

env_path = os.path.join(os.getcwd(), r'N:\MyRL\MyfirstMPC\UnityEnvironment.exe')  # Замените на ваш путь к .exe


engine_channel = EngineConfigurationChannel()
env = UnityEnvironment(file_name=env_path, side_channels=[engine_channel])
env.reset()
# Получение информации о среде
behavior_names = list(env.behavior_specs.keys())
print('Доступные поведения:', behavior_names)

Доступные поведения: ['MyAgent?team=0']


In [ ]:
# Инициализация среды Unity
# env = UnityEnvironment(file_name="path/to/your/environment")  # Укажите путь к вашему файлу среды
# env.reset()

In [3]:
# Пример цикла взаимодействия с агентом
behavior_name = behavior_names[0]  # Используем первое поведение
spec = env.behavior_specs[behavior_name]
decision_steps, terminal_steps = env.get_steps(behavior_name)
for episode in range(1):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    while len(terminal_steps) == 0:
        action = spec.action_spec.random_action(len(decision_steps))
        env.set_actions(behavior_name, action)
        env.step()
        decision_steps, terminal_steps = env.get_steps(behavior_name)
    print(f"Эпизод {episode+1} завершён.")

Эпизод 1 завершён.


In [4]:
# Получение имени поведения
behavior_names = list(env.behavior_specs.keys())
behavior_name = behavior_names[0]
spec = env.behavior_specs[behavior_name]

In [5]:
print(f"Имя поведения: {behavior_name}")
# Получение спецификации поведения
print(f"Спецификация поведения: {spec}")

Имя поведения: MyAgent?team=0
Спецификация поведения: BehaviorSpec(observation_specs=[ObservationSpec(shape=(6,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size6')], action_spec=ActionSpec(continuous_size=2, discrete_branches=(1,)))


In [7]:
# Пример простой нейросети для агента
class SimpleAgentNet(nn.Module):
    def __init__(self, obs_size, action_size):
        super(SimpleAgentNet, self).__init__()
        self.fc1 = nn.Linear(obs_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, action_size)
    
    def forward(self, x):
        # x может быть (obs_size,) или (batch, obs_size)
        if x.dim() == 1:
            x = x.unsqueeze(0)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
# Получаем размеры входа и выхода из спецификации поведения
obs_size = spec.observation_specs[0].shape[0]
action_size = spec.action_spec.continuous_size

# Инициализация сети, оптимизатора и функции потерь
net = SimpleAgentNet(obs_size, action_size)
optimizer = optim.Adam(net.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Гиперпараметры
num_episodes = 100000
gamma = 0.99

print(f"Размер наблюдений: {obs_size}")
print(f"Размер действий: {action_size}")
print(f"Начинаем обучение на {num_episodes} эпизодов...")

Размер наблюдений: 6
Размер действий: 2
Начинаем обучение на 100000 эпизодов...


In [6]:
env.reset()
env.close()  # Закрытие среды после завершения обучения